In [1]:
import pandas as pd
from collections import defaultdict
import collections
import hashlib
import re

In [2]:
#Read Dana's file (headers are simplified)
df = pd.read_csv("output\intermediate\Danas_File_ExtendedWithIds.csv",dtype = str)
print(len(df.index))

1497


<h1>MD5 Hash Generating Function</h1>

In [3]:
# Following operations are performed
# (1) Trim (2) Convert to lower (c) Replace consequitive white space characters with underscore (_)

pattern = re.compile(r'\s+')
def generateIdBasedOnHash(text):
    normalized_text = text.strip().lower()
    normalized_text = re.sub(pattern, '_', normalized_text)
    return str(hashlib.md5(normalized_text.encode('utf-8')).hexdigest())

<h1>Generating Benefit Package</h1>

In [4]:
df_benefit_tier_subset = df[['community_id','Benefit Tier']].drop_duplicates().dropna(subset=['community_id','Benefit Tier'])

In [5]:
# Basic Standardization
df_benefit_tier_subset = df_benefit_tier_subset.apply(lambda x: x.astype(str).str.upper())
df_benefit_tier_subset = df_benefit_tier_subset.apply(lambda x: x.astype(str).str.strip())
df_benefit_tier_subset = df_benefit_tier_subset.apply(lambda x: x.astype(str).str.replace('  ',' '))

In [6]:
d1 = []
for row in df_benefit_tier_subset.itertuples():
    community = row[1]
    benefit_tier = row[2].strip()
    benefit_package_id = generateIdBasedOnHash(community + "|" + benefit_tier)
    d1.append({"community": community, "benefit_package_id": benefit_package_id, "benefit_tier": benefit_tier})
    
benefit_package_df = pd.DataFrame(d1)
benefit_package_df = benefit_package_df[['community','benefit_package_id','benefit_tier']]

<H3>Golf Benefits</H3>

In [7]:
df = df[(~df['Golf Benefit'].isnull())]
df_subset = df[(~df['Golf Benefit'].isin(['No access','Home Club','NO ACCESS','NAN']))]\
                [['community_id','Benefit Tier','Golf Benefit','Availability','Advance','entity_code','Golf Amenity Restrictions']].drop_duplicates().dropna(subset=['community_id','Benefit Tier','Golf Benefit','entity_code'])

In [8]:
# Basic Standardization
df_subset = df_subset.apply(lambda x: x.astype(str).str.upper())
df_subset = df_subset.apply(lambda x: x.astype(str).str.strip())
df_subset = df_subset.apply(lambda x: x.astype(str).str.replace('  ',' '))

In [9]:
d = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(list))))

In [10]:
for row in df_subset.itertuples():
    if row[7] :
        d[row[1]][row[2]][row[3]+ " # " + row[5]][row[4]].append(row[6]+ "##" + row[7])       
    else :  
        d[row[1]][row[2]][row[3]+ " # " + row[5]][row[4]].append(row[6])

In [11]:
communities = list(d.keys())

In [12]:
d2 = []
d3 = []
d4 = []
for community in communities:
    benefit_tiers = list(d[community].keys())
    for benefit_tier in benefit_tiers:
        benefit_package_id_golf = generateIdBasedOnHash(community + "|" + benefit_tier.strip())
        benefits = list(d[community][benefit_tier])
        for benefit in benefits:
            benefit_id = generateIdBasedOnHash(community + "|Golf|" + benefit_tier + "|" + benefit)
            d2.append({"community": community,"benefit_package_id": benefit_package_id_golf, "benefit_id": benefit_id, "benefit": benefit})
            access_rules = list(d[community][benefit_tier][benefit])
            for access_rule in access_rules:
                clubs = list(d[community][benefit_tier][benefit][access_rule])
                if access_rule == 'NAN':
                    access_rule = "Access Rule"
                access_rule_id = generateIdBasedOnHash(community + "|Golf|" + benefit_tier + "|" + benefit + "|" + access_rule)
                d3.append({"benefit_id": benefit_id, "access_rule_id": access_rule_id, "access_rule": access_rule})
                
                for club in clubs:
                    token = club.split("##")
                    entityCode = token[0]
                    restriction = ''
                    if len(token)>1 :
                        restriction = token[1]
                    d4.append({"benefit_id": benefit_id, "access_rule_id": access_rule_id, "access_rule": access_rule, "entity_code":entityCode,"Type" : "Golf", 'Restriction' : restriction})

<b>Golf Data Frames</b>

In [13]:
benefit_df = pd.DataFrame(d2)
benefit_df = benefit_df[['community','benefit_package_id','benefit_id','benefit']]
access_rule_df = pd.DataFrame(d3)
access_rule_df = access_rule_df[['benefit_id','access_rule_id','access_rule']]
access_rule_club_df = pd.DataFrame(d4)
access_rule_club_df = access_rule_club_df[['benefit_id','access_rule_id','access_rule','entity_code', 'Type','Restriction']]

<H3>Dining Benefits</H3>

In [14]:
df = pd.read_csv("output\intermediate\Danas_File_ExtendedWithIds.csv",dtype = str)
print(len(df.index))
df = df[(~df['Dining Benefit'].isnull())]
df_subset_dining = df[(~df['Dining Benefit'].isin(['No access','Home Club','N/A','NO ACCESS','NAN']))]\
                    [['community_id','Benefit Tier','Dining Benefit','Dining Availability','entity_code','Dining Restrictions']].drop_duplicates()
    
    
print(len(df_subset_dining.index))

df_subset_check = df_subset_dining[(df_subset_dining['community_id'].isin(['5345','5347','5348','5349','5315','5338']))]
df_subset_check = df_subset_check[(df_subset_check['Benefit Tier'].isin(['ALL COMMUNITY PROGRAMMING']))]

print(len(df_subset_check.index))

1497
1290
10


In [15]:
# Basic Standardization
df_subset_dining = df_subset_dining.apply(lambda x: x.astype(str).str.upper())
df_subset_dining = df_subset_dining.apply(lambda x: x.astype(str).str.strip())
df_subset_dining = df_subset_dining.apply(lambda x: x.astype(str).str.replace('  ',' '))

In [16]:
d_dining = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(list))))
for row in df_subset_dining.itertuples():
    d_dining[row[1]][row[2]][row[3]][row[4]].append(row[5] +"##" + row[6])

In [17]:
communities = list(d_dining.keys())

d2_dining = []
d3_dining = []
d4_dining = []

list_club = ['5345','5347','5348','5349','5315','5338']


for community in communities:
    #print(community)
    benefit_tiers = list(d_dining[community].keys())
    for benefit_tier in benefit_tiers:
        benefit_package_id_dining = generateIdBasedOnHash(community + "|" + benefit_tier.strip())
        benefits = list(d_dining[community][benefit_tier])
        
        
        if community in list_club :
            print("------------------------------------")
            print(community)
            print(benefit_package_id_dining)
            print(benefits)
            
        for benefit in benefits:
            benefit_id = generateIdBasedOnHash(community + "|Dining|" + benefit_tier + "|" + benefit)
            d2_dining.append({"community": community,"benefit_package_id": benefit_package_id_dining, "benefit_id": benefit_id, "benefit": benefit})
            access_rules = list(d_dining[community][benefit_tier][benefit])
            
            if community in list_club :  
                print(benefit_id)
            
            for access_rule in access_rules:
                clubs = list(d_dining[community][benefit_tier][benefit][access_rule])
                if access_rule == 'NAN':
                    access_rule = "Access Rule"
                
                access_rule_id = generateIdBasedOnHash(community + "|Dining|" + benefit_tier + "|" + benefit + "|" + access_rule)
                d3_dining.append({"benefit_id": benefit_id, "access_rule_id": access_rule_id, "access_rule": access_rule})
                
                for club in clubs:          
                    token = club.split("##")
                    entityCode = token[0]
                    restriction = ''
                    if len(token)>1 :
                        restriction = token[1]                  
                    d4_dining.append({"benefit_id": benefit_id, "access_rule_id": access_rule_id, "access_rule": access_rule, "entity_code":entityCode,"Type" : "Dining", "Restriction" : restriction})


------------------------------------
5338
050e2a6d5c64a8e416673096d42bf16c
['50% OFF TWO MEMBER MEALS PER CLUB', 'LUNCH ACCESS']
d56dc079ad791b356e44f96fdf899fb4
120cd73056daa6f473aabd645401b4e9
------------------------------------
5345
99f39b71f4350e20f95df39e456b8f39
['50% OFF TWO MEMBER MEALS PER CLUB', 'LUNCH ACCESS']
05496543edf35b70b47334a2fe50bfea
aec6fffb6fa88a6fedc72d847e432d67
------------------------------------
5349
5fc107a38e2867820696fea9415a0cf2
['TWO 2-4-1 PER CLUB DINING', 'ADDITIONAL PAY TO DINE ACCESS AVAILABLE']
935f0d9ce25e44f371bb672cca274c7b
f38f55b12a753ac667cec58ab4777220
------------------------------------
5315
0e4a3fec96524e29fde3df05267eed3e
['ONE 2-4-1 PER CLUB DINING', 'ADDITIONAL PAY TO DINE ACCESS AVAILABLE']
12fda5120d1c85ba50eceb0640740881
8efc71d2a08cf93427aa3ea670d8e580
------------------------------------
5348
1cc73dd645f7668b95eabce233a499f1
['PAY TO DINE']
8c50501c7eb4f0573a2bff5869d445a8
------------------------------------
5347
33cb606383950b4a

<b>Dining Data Frames</b>

In [18]:
dining_benefit_df = pd.DataFrame(d2_dining)
dining_benefit_df = dining_benefit_df[['community','benefit_package_id','benefit_id','benefit']]
dining_access_rule_df = pd.DataFrame(d3_dining)
dining_access_rule_df = dining_access_rule_df[['benefit_id','access_rule_id','access_rule']]
dining_access_rule_club_df = pd.DataFrame(d4_dining)
dining_access_rule_club_df = dining_access_rule_club_df[['benefit_id','access_rule_id','access_rule','entity_code','Type', 'Restriction']]

<B> Merging two Benefit Package generated for Golf and Dining</B>

<b>Save Files</b>

In [19]:
benefit_package_df.to_csv("output\Community_Benefit_Packages.csv", index=False)
benefit_df.to_csv("output\intermediate\Community_Golf_Benefits.csv", index=False)
access_rule_df.to_csv("output\intermediate\Community_Golf_Access_Rules.csv", index=False)
access_rule_club_df.to_csv("output\Community_Golf_Access_Rules_Club.csv", index=False)
dining_benefit_df.to_csv("output\intermediate\Community_Dining_Benefits.csv", index=False)
dining_access_rule_df.to_csv("output\intermediate\Community_Dining_Access_Rules.csv", index=False)
dining_access_rule_club_df.to_csv("output\Community_Dining_Access_Rules_Club.csv", index=False)